In [3]:
import pandas as pd

start_year = 2023
start_month = 1
end_year = 2024
end_month = 1
date1 = f"{start_year}-{str(start_month).zfill(2)}-01"
date2 = f"{end_year}-{str(end_month).zfill(2)}-01"

# Создание списка ['2023-01-01', '2023-02-01', ....
month_list = pd.date_range(date1, date2,
                           freq="D").strftime("%Y-%m-%d").tolist()

In [4]:
def code_condition(codes: list):
    match len(codes):
        case 1:
            return f"CODE = '{codes[0]}'"
        case 2:
            return f"CODE = '{codes[0]}' OR CODE = '{codes[1]}'"
        case 3:
            return f"CODE = '{codes[0]}' OR CODE = '{codes[1]}' OR CODE = '{codes[2]}'"
        case 4:
            return f"CODE = '{codes[0]}' OR CODE = '{codes[1]}' OR CODE = '{codes[2]}' OR CODE = '{codes[3]}'"
        case 5:
            return f"CODE = '{codes[0]}' OR CODE = '{codes[1]}' OR CODE = '{codes[2]}' OR CODE = '{codes[3]}' OR CODE = '{codes[4]}'"

In [5]:
def prepare_query(
        date1: float | int,
        date2: float | int,
        lat1: float | int,
        lat2: float | int,
        lon1: float | int,
        lon2: float | int,
        codes: list[str]
) -> str:
    selectquery = f"""
select 
	`code`,
	`date`,
	`latitude`,
	`longitude`,
	`radius`,
	`v1`,
	`v2`,
	`v3`,
	`n`,
	`e`,
	`c`,
	`f`,
	`dst`,
    `kp`

from (select * from `geomag`.`sat_sec_plain`
	left join `geomag`.`index_view` on 
	`geomag`.`sat_sec_plain`.`date` = `geomag`.`index_view`.`index_date`) `iv`
	
	where `iv`.`date` > {date1} and `iv`.`date` <= {date2}
	and
	`iv`.`latitude` >= {lat1} and `iv`.`latitude` <= {lat2}
	and
	`iv`.`longitude` >= {lon1} and `iv`.`longitude` <= {lon2}
	and
    {code_condition(codes)}
                
                """

    return selectquery

In [37]:
from datetime import datetime


def download_data_from_sql(date1, date2, dbConnect, codes, lat1=-90, lat2=+90, long1=-180, long2=+180):
    d1 = datetime.strptime(date1, "%Y-%m-%d").timestamp()
    d2 = datetime.strptime(date2, "%Y-%m-%d").timestamp()

    # selectquery = prepare_query(d1, d2, lat1, lat2, long1, long2, code)
    frame = pd.read_sql(
        f"""
        select 
	`code`,
	`date`,
	`latitude`,
	`longitude`,
	`radius`,
	`v1`,
	`v2`,
	`v3`,
	`n`,
	`e`,
	`c`,
	`f`
    from `geomag`.`sat_sec_plain`
    where `date` between {d1} and {d2}
	""", dbConnect)
    frame.to_csv("test.csv", index=False)
    return frame

In [38]:
from sqlalchemy import create_engine
ip = '127.0.0.1:3306'
password = 'root'
login = 'root'


sqlEngine = create_engine(f"mysql+pymysql://{login}:{password}@{ip}", pool_recycle=3600)
print("Подключение создано")

# Подключаемся к БД
dbConnection = sqlEngine.connect()
print("Подключение установлено")


satellites = ["SWA", "SWB"]

# Загружаем данные в DataFrame
print("Перед результатом")
result = download_data_from_sql(month_list[0], month_list[1], dbConnection, satellites)
dbConnection.close()
print("После результата")

Подключение создано
Подключение установлено
Перед результатом


KeyboardInterrupt: 

In [8]:
month_list[1]

'2023-01-02'

In [32]:

dbConnection.close()